In [ ]:
#주요 라이브러리 설치
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers
!pip install pytorch-lightning

#KoBERT Tokenizer 파일 다운로드
!wget https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 2.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 23.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595745 sha256=78418ec72e1b4ab43efdb0683c454a9dc2350b91dc5095ae9ca0b4487ddd75ce
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 30.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [ ]:
import torch

from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

In [ ]:
from transformers import BertModel
bert_embedding_model = BertModel.from_pretrained('monologg/kobert')

from tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test2.csv to test2.csv


In [ ]:
TrainData = pd.read_csv('train.csv')
TestData = pd.read_csv('test2.csv')

TrainData['title'] = TrainData['title'].str.replace("[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
TrainData['title'] = TrainData['title'].str.replace(" +"," ")

TestData['title'] = TestData['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
TestData['title'] = TestData['title'].str.replace(" +"," ")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
TrainData['title']

0                                  유망 벤처기업 투자자 연결
1                                   출범이후 첫 교육청 감사
2                              영화 음반 등 5대산업 중점 육성
3                                   경남고 봉황대기 16강에
4                                     허정무 화끈한 출항 
                           ...                   
10995           오늘의 중도일보 뉴스 대전 세종 충남 충북 6월6일 화요일 
10996                           기고 노예와 로봇 그리고 일자리
10997             한국철도협 철도인 신년교례회 성료 해외 진출 적극 지원 
10998    서울대 총장의 탄식 4차산업혁명 선도해야할 한국의 대학들은 되레 역주행 
10999                         경산시 미래형 자동차산업 육성 시동
Name: title, Length: 11000, dtype: object

In [ ]:

# TrainData = TrainData.dropna(axis=0)

In [ ]:
BATCH_SIZE = 8

#tokenizer로 토큰화
train_tokens = tokenizer.batch_encode_plus(TrainData['title'].values, padding=True, truncation=True, max_length=512, return_tensors='pt')
test_tokens = tokenizer.batch_encode_plus(TestData['title'].values, padding=True, truncation=True, max_length=512, return_tensors='pt')

#input_ids, attention_mask, label을 묶어서 하나의 데이터 셋으로 구축
train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], torch.LongTensor(TrainData['label'].values))
test_dataset = TensorDataset(test_tokens['input_ids'], test_tokens['attention_mask'], torch.LongTensor(TestData['label'].values))

#train data의 10%를 validation data로 구축
train_dataset, validation_dataset = train_test_split(train_dataset, test_size = 0.1)

#데이터를 batch로 나누어 DataLoader로 변환
train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset,batch_size = BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset,batch_size = BATCH_SIZE, shuffle=False)

In [ ]:
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score

class KoBERTmodel(pl.LightningModule):
    def __init__(self,model, num_labels):
        super(KoBERTmodel, self).__init__()
        self.bert = model
        self.dropout = torch.nn.Dropout(p=0.1,inplace=False)
        self.classifier = torch.nn.Linear(768,num_labels)
        
        self.loss = torch.nn.CrossEntropyLoss()

    # 정확도 계산 함수
    def flat_accuracy(self, preds, labels):
        preds = preds.to('cpu').detach().numpy()
        labels = labels.to('cpu').detach().numpy()
        return accuracy_score(preds, labels)

    def forward(self, input_ids, attention_mask, **kwargs):
        x = self.bert(input_ids = input_ids, attention_mask = attention_mask)
        x = self.dropout(x['pooler_output'])
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, label = batch
        logits = self(input_ids = input_ids, attention_mask = attention_mask)
        loss = self.loss(logits, label)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, label = batch
        logits = self(input_ids = input_ids, attention_mask = attention_mask)
        preds = logits.argmax(dim=1)
        acc = self.flat_accuracy(preds, label)
        loss = self.loss(logits, label)
        metrics = {'val_acc': acc, 'val_loss': loss}
        self.log_dict(metrics)

    def test_step(self, batch, batch_idx):
        input_ids, attention_mask, label = batch
        logits = self(input_ids = input_ids, attention_mask = attention_mask)
        preds = logits.argmax(dim=1)
        acc = self.flat_accuracy(preds, label)
        loss = self.loss(logits, label)
        metrics = {'test_acc': acc, 'test_loss': loss}
        self.log_dict(metrics)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=2e-5, eps = 1e-8)
        return optimizer

In [ ]:
clf_model = KoBERTmodel(bert_embedding_model, 2)

In [ ]:

trainer = pl.Trainer(max_epochs=5, gpus=1)
trainer.fit(clf_model, train_loader, validation_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | bert       | BertModel        | 92.2 M
1 | dropout    | Dropout          | 0     
2 | classifier | Linear           | 1.5 K 
3 | loss       | CrossEntropyLoss | 0     
------------------------------------------------
92.2 M    Trainable params
0         Non-trainable params
92.2 M    Total params
368.754   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
trainer.test(dataloaders=test_loader)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1447: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at /content/lightning_logs/version_0/checkpoints/epoch=4-step=6190.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /content/lightning_logs/version_0/checkpoints/epoch=4-step=6190.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                   0.98
        test_loss           0.07219985127449036
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.98, 'test_loss': 0.07219985127449036}]

In [ ]:
# 임의로 기사제목을 넣어서 테스트가 가능합니다.

test_sentences = ['통계학과 SBO 팀 화이팅','백신 부작용 여름에는 더 심해짐','‘그알’ 음원 사재기 의혹 후폭풍…바이브·닐로 쪽 반격 나서', '아직 재유행 아니지만 가을 대비 개량 백신 확충 필요']
test_input = tokenizer.batch_encode_plus(test_sentences, padding=True, truncation=True, max_length=512, return_tensors='pt')
test_input

{'input_ids': tensor([[   2, 4744, 7822,  687,  272, 4797, 5112, 7096, 7681,    3,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   2, 2298, 6733, 2448, 3307, 6900, 1698, 3060, 7848, 7353,    3,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   2,  712, 5538, 6821,  498, 3611, 2573, 7191, 5561, 3644, 5176, 7734,
         7750,   55, 6277, 6432,  478, 5777, 6079, 4398, 2207, 5412, 1381,    3],
        [   2, 3129, 3969, 7063, 7881, 3104,  517, 5330, 7088, 1649,  835, 6035,
         2298, 6733, 5135, 4919,    3,    1,    1,    1,    1,    1,    1,    1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
print(test_sentences)
result = clf_model(**test_input)
print(result.argmax(dim=1))

['통계학과 SBO 팀 화이팅', '백신 부작용 여름에는 더 심해짐', '‘그알’ 음원 사재기 의혹 후폭풍…바이브·닐로 쪽 반격 나서', '아직 재유행 아니지만 가을 대비 개량 백신 확충 필요']
tensor([1, 0, 0, 0])


In [ ]:
!nvidia-smi

Mon Jul  4 09:59:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    40W /  70W |   1428MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------